# Set up OAuth APIs with tls_client_auth
Running this script will create the APIs so we can sign JWTs and use the private_key_jwt authorization methods

## Requirements
1. Create an APImetrics Project
1. Get an API key with EDITOR permissions for the project, enter it below
1. Run this workbook to create all Auth Settings, APIs and Workflows for Sign-in flow
1. Set environment variables:
    1. kid_cert_version - this is the numeric ID internal to APImetrics for your signing key
    1. kid - this is your signing certs real key ID (as per JWKS)
    1. client_id - this is your client_id 
    1. issuer - this is the issuer for your JWTs
    1. redirect_uri - this is your redirect_uri for OAuth
    
NB: Until we publish the API call conditions API, those need to be added manually from the conditions tab

In [47]:
# Enter intended project's APImetrics API key here:
API_KEY = input("Enter your APImetrics API key for the intended project: ")

Enter your APImetrics API key for the intended project: mTkWg8NQKURDgzNmEbnf4yhem51ZgWqF


In [48]:
# Put your .well-known JSON here
import json
true = True # to make copy-pasteing easier 
false = False # to make copy-pasteing easier 
WELL_KNOWN = None
if not WELL_KNOWN:
    WELL_KNOWN = json.loads(input("Enter your .well-known JSON here: "))

Enter your .well-known JSON here:            { 	"issuer": "https://api.nationwide.co.uk/open-banking", 	"authorization_endpoint": "https://api.obtpp.nationwideinterfaces.io/open-banking/oauth/authorize", 	"token_endpoint": "https://api.obtpp.nationwideinterfaces.io/open-banking/oauth/token", 	"jwks_uri": "https://keystore.openbanking.org.uk/0015800000jf8aKAAQ/bghrOqZUMgBTV07eFcydf.jwks", 	"registration_endpoint": "https://api.obtpp.nationwideinterfaces.io/open-banking/register", 	"scopes_supported": ["openid", "accounts", "payments", "fundsconfirmations"], 	"claims_supported": ["openbanking_intent_id"], 	"response_types_supported": ["code id_token"], 	"grant_types_supported": ["authorization_code", "refresh_token", "client_credentials"], 	"subject_types_supported": ["public"], 	"id_token_signing_alg_values_supported": ["PS256"], 	"token_endpoint_auth_methods_supported": ["tls_client_auth"], 	"request_parameter_supported": true, 	"claims_parameter_supported": true, 	"request_object_sign

In [49]:
# Enter your intended scope for your tokens
scopes_in = input("Enter the scopes you wish to use (except openid, we'll add that automatically): ")
scopes = ['openid'] + [scope.strip() for scope in scopes_in.split(' ')]
for scope in scopes:
    assert scope in WELL_KNOWN['scopes_supported'], f"Scope {scope} is not supported according to .well-known"
SCOPES = " ".join(scopes)
print(f"Using scopes {SCOPES}")

Enter the scopes you wish to use (except openid, we'll add that automatically): accounts
Using scopes openid accounts


In [50]:
assert 'tls_client_auth' in WELL_KNOWN['token_endpoint_auth_methods_supported'], 'This script is for private_key_jwt but your well-known does not claim that it is supported'

# We now extract the values we need (or you can override it)
AUTHORIZATION_URL = WELL_KNOWN['authorization_endpoint']
ACCESS_TOKEN_URL = WELL_KNOWN['token_endpoint']
DOMAIN = input("Domain for Bank APIs: ")

Domain for Bank APIs: api.obtpp.nationwideinterfaces.io


In [51]:
# Helper functions
import requests
import json
import urllib
from apimetrics_api import APImetricsAPI 
    
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

# [CLIENT.delete_token(o['id']) for o in CLIENT.tokens.values()]
# [CLIENT.delete_auth(o['id']) for o in CLIENT.auths.values()]
# [CLIENT.delete_call(o['id']) for o in CLIENT.calls.values()]
# [CLIENT.delete_workflow(o['id']) for o in CLIENT.workflows.values()]

From here on out, you can pick and choose what you want to run.


## Create Auth Setting for APImetrics API

In [52]:
# First, create APImetrics API Auth Setting
tag = 'auth:apimetrics_api'
if tag not in CLIENT.auths_by_tag:
    setup = {
        "meta": {
            "domain": "client.apimetrics.io",
            "documentation": {
                "keys": "https://client.apimetrics.io/settings/api-key",
                "docs": "https://apimetrics.readme.io/v2/reference",
                "apps": "",
                "provider": "https://client.apimetrics.io/",
            },
            "name": "APImetrics API",
            "tags": [tag],
            "description": "API that allows you to call APImetrics' API.",
        },
        "settings": {"auth_type": "MANUAL"}
    }
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
else:
    print(f"Found Auth Setting {CLIENT.auths_by_tag[tag]}")

# Next, Create Token
if CLIENT.auths_by_tag[tag] not in CLIENT.tokens_by_auth:
    setup = {
        'meta': {
            'name': 'Project Access Token',
            'domain': 'client.apimetrics.io',
            'auth_id': CLIENT.auths_by_tag[tag]
        },
        'token': {
            'headers': [
              {
                'p_key': 'Authorization',
                'p_val': f'Bearer {API_KEY}',
              },
            ],
        }
    }
    token = CLIENT.create_token(setup)
    print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")

Found 5 Auths
Auth tags: auth:apimetrics_api, auth:bank_matls
Found Auth Setting agpzfnZpYXRlc3RzchoLEg1TZXJ2aWNlQ29uZmlnGICA6Megy5EJDA
Found 5 Tokens
Tokens for auths: APImetrics API, Pyxis Transport MATLS, Transport MATLS, APImetrics API, Pyxis JWT Signer


## Create Auth for calling Bank APIs

In [53]:
# First create Auth Setting
tag = 'auth:bank_matls'
if tag not in CLIENT.auths_by_tag:

    ssl_key = None
    ssl_cert = None

#     if SSL_KEY_PATH:
#         with open(SSL_KEY_PATH) as stream:
#             ssl_key = stream.read()

#     if SSL_CERT_PATH:
#         with open(SSL_CERT_PATH) as stream:
#             ssl_cert = stream.read()
    
    setup = {
        "access": {
            "keys": False,
            "org_keys": False,
            "org_settings": True,
            "settings": False,
        },
        "keys": {},
        "meta": {
            "domain": DOMAIN,
            "documentation": {"keys": "", "docs": "", "apps": "", "provider": ""},
            "name": "Transport MATLS",
            "tags": [tag],
            "description": "Mutual Authenticated TLS for calls to bank APIs",
        },
        "settings": {
            "auth_type": "MANUAL",
            "ssl_key": ssl_key,
            "ssl_cert": ssl_cert,
        },
    }
#     if OAUTH_METHOD == "client_secret_post" or OAUTH_METHOD == "client_secret_basic":
#         setup["keys"]["client_id"] = CLIENT_ID
#         setup["keys"]["client_secret"] = CLIENT_SECRET
    
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
    
# Second Create Token
if CLIENT.auths_by_tag['auth:bank_matls'] not in CLIENT.tokens_by_auth:
    setup = {
        'meta': {
            'name': 'Authenticated User Access Token',
            'domain': DOMAIN,
            'auth_id': CLIENT.auths_by_tag['auth:bank_matls']
        },
        'token': {}
    }
    token = CLIENT.create_token(setup)
    print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")


## Create APImetrics API helpers for setting env variables

In [54]:
def get_set_env(tag, var_name, value):
    body = {"value": value}
    body_str = json.dumps(body)

    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:update", "sector:devtools", tag],
            "name": "APImetrics: Save {}".format(var_name),
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "https://client.apimetrics.io/api/2/environment/global/{}".format(
                var_name
            ),
            "auth_id": CLIENT.auths_by_tag['auth:apimetrics_api'],
            "headers": [
                {"value": "application/json", "key": "Accept"},
                {"value": "application/json", "key": "Content-Type"},
            ],
            "token_id": CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:apimetrics_api']],
            "method": "POST",
        },
    }
    return setup

In [55]:
tag = 'apim:set_env:code'
if tag not in CLIENT.calls_by_tag:
    setup = get_set_env(tag, 'code', '__CODE__')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

Found 16 Calls
Call tags: api_type:update, sector:devtools, apim:set_env:code, apim:set_env:refresh_token, apim:tokens:update, api_type:create, sector:financial, banks:3.1:dynamic_registration:create, ob_id:0, banks:oauth:code, banks:oauth:client_credentials, banks:oauth:refresh_token, banks:oauth:authorize, jwt:sign:dynamic_registration, api_expert:exclude, jwt:sign:authorize, ob_id:1, ob_v:3.1, banks:3.1:account-access-consents:create_max, banks:3.1:account-access-consents:create_min, api_type:delete, ob_id:3, banks:3.1:account-access-consents:delete, api_type:read, ob_id:2, banks:3.1:account-access-consents:read


In [56]:
tag = 'apim:set_env:refresh_token'
if tag not in CLIENT.calls_by_tag:
    setup = get_set_env(tag, 'refresh_token', '__REFRESH_TOKEN__')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## Create APImetrics API helper to save token

In [57]:
tag = 'apim:tokens:update'
if tag not in CLIENT.calls_by_tag:
    body = {
        "token": {
            "headers": [
                {"p_key": "Authorization", "p_val": "Bearer %%ACCESS_TOKEN%%"},
            ],
            "expires_in": "%%EXPIRES_IN%%",
        }
    }
    body_str = json.dumps(body, indent=2)
    body_str = body_str.replace('"%%EXPIRES_IN%%"', "%%EXPIRES_IN%%")

    setup = {
        "meta": {
            "tags": ["api_type:update", "sector:devtools", tag],
            "name": "APImetrics: Update Auth Token",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "https://client.apimetrics.io/api/2/tokens/{{apim_token_id}}/",
            "auth_id": CLIENT.auths_by_tag['auth:apimetrics_api'],
            "headers": [{"value": "application/json", "key": "Content-Type"}],
            "token_id": CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:apimetrics_api']],
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")
    
    # Set environment variable we use in the API
    CLIENT.set_env_variable(
        'global', 
        'apim_token_id', 
        CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:bank_matls']])

### OAuth Authorize Claims - build JWT and sign it

In [58]:
# For user authentication
tag = 'jwt:sign:authorize'
if tag not in CLIENT.calls_by_tag:
    auth_tag = 'auth:apimetrics_api'
    auth_id = CLIENT.auths_by_tag[auth_tag]
    token_id = CLIENT.tokens_by_auth[auth_id]
    
    body = {
        "header": {},
        "payload": {
            "scope": SCOPES,
            "claims": {
                "id_token": {
                    "acr": {"value": "urn:openbanking:psd2:sca", "essential": True},
                    "openbanking_intent_id": {
                        "value": "%%CONSENT_ID%%",
                        "essential": True,
                    },
                },
                "userinfo": {
                    "openbanking_intent_id": {
                        "value": "%%CONSENT_ID%%",
                        "essential": True,
                    }
                },
            },
            "response_type": "code id_token",
            "redirect_uri": "{{redirect_uri}}",
            "state": "__TEST_RUN_RESULT_ID__",
            "nonce": "__TEST_RUN_RESULT_ID__",
            "client_id": "{{client_id}}",
        },
        "private_key": {
            "versionId": '{{kid_cert_version}}'
        }
    }
    
    body_str = json.dumps(body, indent=2)
    
    setup = {
        "meta": {
            "tags": ["api_type:create", "sector:devtools", tag],
            "name": "Sign User Auth JWT with KMS Cert",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [
                {"value": "{{kid}}", "key": "kid"},
                {"value": "RS256", "key": "alg"},
                {"value": "{{issuer}}", "key": "aud"},
                {"value": "{{client_id}}", "key": "iss"},
                {"value": "10m", "key": "expiresIn"},
            ],
            "url": "https://us-central1-viatests.cloudfunctions.net/jwt-kms-signer",
            "auth_id": auth_id,
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "Content-Type", "value": "application/json"},
            ],
            "token_id": token_id,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## OAuth - API calls
### Client Credentials flow

In [59]:
tag = 'banks:oauth:client_credentials'
if tag not in CLIENT.calls_by_tag:
    
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]
    
    params = {
        "grant_type": "client_credentials",
        "scope": SCOPES,
        "client_id": "{{client_id}}",
    }
    params_str = urllib.parse.urlencode(params)
    
    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: client_credentials",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Auth Code flow - go to user authorization page

In [60]:
tag = 'banks:oauth:authorize'
if tag not in CLIENT.calls_by_tag:
    
    headers = [{"value": "*/*", "key": "Accept"}]
    
    params = {
        "response_type": "code id_token",
        "scope": SCOPES,
        "client_id": "{{client_id}}",
        "state": "__TEST_RUN_RESULT_ID__",
        "nonce": "__TEST_RUN_RESULT_ID__",
        "redirect_uri": "{{redirect_uri}}",
        "request": "__JWT_TOKEN__",
    }
    
    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Authorize",
            "workspace": "global",
        },
        "request": {
            "body": None,
            "parameters": params,
            "url": AUTHORIZATION_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "GET",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Auth Code flow - get access token

This is the call that swaps the temporary code from the user sign in redirect URL for an access token

In [61]:
# For auth_code flow
tag = 'banks:oauth:code'
if tag not in CLIENT.calls_by_tag:
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]

    params = {
        "grant_type": "authorization_code",
        "code": "{{code}}",
        "redirect_uri": "{{redirect_uri}}",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": "__JWT_TOKEN__",
    }
    params_str = urllib.parse.urlencode(params)

    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: authorization_code",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Refresh token flow

In [62]:
# For auth_code flow
tag = 'banks:oauth:refresh_token'
if tag not in CLIENT.calls_by_tag:
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]

    params = {
        "grant_type": "refresh_token",
        "refresh_token": "{{refresh_token}}",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": "__JWT_TOKEN__",
    }
    params_str = urllib.parse.urlencode(params)

    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: refresh_token",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## Create Consent APIs to use

Go to [OBIE - Account Consent APIs.ipynb](./OBIE%20-%20Account%20Consent%20APIs.ipynb) to create the consent APIs

In [63]:
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

## Create Workflows to for User sign-in (authorize_code flow)

In [64]:
# The user sign-in flow step 1
tag = 'banks:oauth:flow:authorize_code:1'
if tag not in CLIENT.workflows_by_tag:
            
    # This is the API that generates the claims you're going to use in the Auth
    claims_tag = 'banks:3.1:account-access-consents:create_max'
    
    call_tags = [
        'banks:oauth:client_credentials',
        claims_tag,
        'jwt:sign:authorize',
        'banks:oauth:authorize',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"
    
    setup = {
      "meta": {
        "name": '[Manual] OAuth: Sign in as User (1/2)', 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    call = CLIENT.create_workflow(setup)
    print(f"Created Workflow {call['meta']['name']} with id {call['id']}")


Found 3 Workflows
Call tags: banks:3.1:account-access-consents:lifecycle, banks:3.1:dynamic_registration
Found 16 Calls
Call tags: api_type:update, sector:devtools, apim:set_env:code, apim:set_env:refresh_token, apim:tokens:update, api_type:create, sector:financial, banks:3.1:dynamic_registration:create, ob_id:0, banks:oauth:code, banks:oauth:client_credentials, banks:oauth:refresh_token, banks:oauth:authorize, jwt:sign:dynamic_registration, api_expert:exclude, jwt:sign:authorize, ob_id:1, ob_v:3.1, banks:3.1:account-access-consents:create_max, banks:3.1:account-access-consents:create_min, api_type:delete, ob_id:3, banks:3.1:account-access-consents:delete, api_type:read, ob_id:2, banks:3.1:account-access-consents:read
Created Workflow [Manual] OAuth: Sign in as User (1/2) with id agpzfnZpYXRlc3RzchQLEgdUZXN0UnVuGICA6MeVzrAKDA


In [65]:
# The user sign-in flow step 2
tag = 'banks:oauth:flow:authorize_code:2'
if tag not in CLIENT.workflows_by_tag:
            
    call_tags = [
        'apim:set_env:code',  # Save OAuth Code
        'banks:oauth:code',
        'apim:set_env:refresh_token',
        'apim:tokens:update',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"
    
    setup = {
      "meta": {
        "name": "[Manual] OAuth: Sign in as User (2/2)", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    call = CLIENT.create_workflow(setup)
    print(f"Created Workflow {call['meta']['name']} with id {call['id']}")


Found 4 Workflows
Call tags: banks:3.1:account-access-consents:lifecycle, banks:3.1:dynamic_registration, banks:oauth:flow:authorize_code:1
Created Workflow [Manual] OAuth: Sign in as User (2/2) with id agpzfnZpYXRlc3RzchQLEgdUZXN0UnVuGICA6KeQlpcLDA


## Create Refresh token workflow

In [66]:
# A workflow to refresh our access token
tag = 'banks:oauth:flow:refresh_token'
if tag not in CLIENT.workflows_by_tag:
    call_tags = [
        'banks:oauth:refresh_token',
        'apim:tokens:update',
        'apim:set_env:refresh_token',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"

    setup = {
      "meta": {
        "name": "OAuth: Refresh token", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    workflow = CLIENT.create_workflow(setup)
    print(f"Created Workflow {workflow['meta']['name']} with id {workflow['id']}")

Found 5 Workflows
Call tags: banks:3.1:account-access-consents:lifecycle, banks:3.1:dynamic_registration, banks:oauth:flow:authorize_code:1, banks:oauth:flow:authorize_code:2
Created Workflow OAuth: Refresh token with id agpzfnZpYXRlc3RzchQLEgdUZXN0UnVuGICA6Oe03P0KDA
